# Lab 16: Advanced Chain Composition with RunnablePassthrough

This lab demonstrates advanced chain composition using RunnablePassthrough for content generation workflows. You'll learn:
- How to use `RunnablePassthrough` for data flow management
- Building multi-step content generation pipelines
- Chaining dependent operations (title → outline → blog → summary)
- Managing data transformation between chain steps
- Creating complex workflows with intermediate results

In [ ]:
# Import LangChain components for advanced content generation workflow
# RunnablePassthrough enables sophisticated data flow management between chain steps
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# Set up OpenAI API credentials for content generation workflow
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [ ]:
# Create the first stage: title generation chain
# RunnablePassthrough() wraps the output in a dictionary with key "title"
# This prepares the data format for the next chain in the sequence
title = (
    ChatPromptTemplate.from_template(
        "Generate an impactful title for {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"title": RunnablePassthrough()}
)

In [ ]:
# Create the second stage: outline generation chain
# Takes the title from previous step and generates a detailed outline
# Wraps output in "outline" key for the next chain step
outline = (
    ChatPromptTemplate.from_template(
        "Generate a detailed outline for {title}"
    )
    | ChatOpenAI()
    | StrOutputParser()
    | {"outline": RunnablePassthrough()}
)

In [ ]:
# Create the third stage: blog post generation chain
# Uses the outline to generate a 200-word blog post
# Wraps output in "blog" key for final summarization step
blog = (
    ChatPromptTemplate.from_template(
        "Generate a 200 word blog post based on the outline: {outline}"
    )
    | ChatOpenAI()
    | StrOutputParser()
    | {"blog": RunnablePassthrough()}
)

In [ ]:
# Create the final stage: summary generation chain
# Takes the full blog post and creates a concise summary
# No RunnablePassthrough wrapper needed as this is the final output
summary = (
    ChatPromptTemplate.from_template(
        "Generate a summary for the post {blog}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [ ]:
# Combine all stages into a complete content generation workflow
# Flow: input → title → outline → blog → summary
# Each stage feeds its output to the next stage using RunnablePassthrough data formatting
content_chain = title | outline | blog | summary

In [ ]:
# Execute the complete content generation workflow
# Input: topic → Output: final summary after going through all stages
# The chain automatically handles data flow between all four stages
content_chain.invoke({"input":"The impact of AI on jobs"})